# foco_queim_inc_silver

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when
from pyspark.sql.types import DoubleType, StringType, IntegerType, TimestampType
from pyspark.sql.types import NumericType

In [0]:
# ==============================
# WIDGETS / PARÂMETROS
# ==============================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_in", "")
dbutils.widgets.text("table_in", "")
dbutils.widgets.text("schema_out", "")
dbutils.widgets.text("table_out", "")
dbutils.widgets.text("data_ref_carga", "")

catalog = dbutils.widgets.get("catalog").strip()
schema_in = dbutils.widgets.get("schema_in").strip()
table_in = dbutils.widgets.get("table_in").strip()
schema_out = dbutils.widgets.get("schema_out").strip()
table_out = dbutils.widgets.get("table_out").strip()
data_ref_carga = dbutils.widgets.get("data_ref_carga").strip()

In [0]:
# ==============================
# TRATAMENTO DE ERROS: parâmetros obrigatórios
# ==============================
params = {
    "catalog": catalog,
    "schema_in": schema_in,
    "table_in": table_in,
    "schema_out": schema_out,
    "table_out": table_out,
    "data_ref_carga": data_ref_carga
}
missing = [k for k, v in params.items() if v == ""]
if missing:
    raise ValueError(f"Parâmetros obrigatórios não informados: {', '.join(missing)}")

In [0]:
# ==============================
# Formatacao de tabela
# ==============================
spark = SparkSession.builder.getOrCreate()
tabela_bronze = f"{catalog}.{schema_in}.{table_in}"
tabela_silver = f"{catalog}.{schema_out}.{table_out}"

print(f"Lendo tabela bronze: {tabela_bronze} - partição data_ref_carga = {data_ref_carga}")

In [0]:
# ==============================
# LEITURA (filtrando por partição)
# ==============================
try:
    df = spark.read.table(tabela_bronze).filter(col("data_ref_carga") == lit(data_ref_carga))
except Exception as e:
    raise RuntimeError(f"Erro ao ler a tabela bronze {tabela_bronze}: {e}")

In [0]:
# ==============================
# TRATAMENTO DE NULOS
# ==============================

In [0]:
# detectar colunas por tipo
string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]
numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, NumericType)]

# aplicar transformações
for c in string_cols:
    df = df.withColumn(c, when(col(c).isNull(), lit(" ")).otherwise(col(c)))

for c in numeric_cols:
    df = df.withColumn(c, when(col(c).isNull(), lit(0)).otherwise(col(c)))

In [0]:
# ==============================
# REGRAS DE VALIDAÇÃO / CORREÇÃO
# - risco_fogo: valor inválido -999 -> 0 (mantemos os outros valores)
# - lat: válido entre -90 e 90 -> caso inválido setar 0
# - lon: válido entre -180 e 180 -> caso inválido setar 0
# ==============================
df = df.withColumn("risco_fogo", when(col("risco_fogo") == -999, lit(0)).otherwise(col("risco_fogo")))
df = df.withColumn("lat", when((col("lat") >= -90.0) & (col("lat") <= 90.0), col("lat")).otherwise(lit(0.0)))
df = df.withColumn("lon", when((col("lon") >= -180.0) & (col("lon") <= 180.0), col("lon")).otherwise(lit(0.0)))

In [0]:
# ==============================
# DEFINIÇÃO DOS TIPOS E COMENTÁRIOS PARA A TABELA SILVER
# ==============================
columns_definition = [
    ("id", "STRING", "Código único"),
    ("lat", "DOUBLE", "Latitude do centro do píxel de fogo ativo apresentada em unidade de graus decimais"),
    ("lon", "DOUBLE", "Longitude do centro do píxel de fogo ativo apresentada em unidade de graus decimais"),
    ("data_hora_gmt", "TIMESTAMP", "Horário de referência da passagem do satélite segundo o fuso horário de Greenwich (GMT). Formato: YYYY-MM-DDTHH:MM:SS.sss+00:00"),
    ("satelite", "STRING", "Nome do algoritmo utilizado e referência ao satélite provedor da imagem."),
    ("municipio", "STRING", "Nome do município. Para o Brasil foi utilizado como referência o dado do IBGE 2000."),
    ("estado", "STRING", "Nome do estado (nível 1 do GADM)."),
    ("pais", "STRING", "Nome do País (nível 0 do GADM)."),
    ("municipio_id", "INT", "Código/ID do município (referência IBGE quando aplicável)."),
    ("estado_id", "INT", "Código do estado"),
    ("pais_id", "INT", "Código do país"),
    ("numero_dias_sem_chuva", "INT", "Número de dias sem chuva até a detecção do foco."),
    ("precipitacao", "DOUBLE", "Valor da precipitação acumulada no dia até o momento da detecção do foco."),
    ("risco_fogo", "DOUBLE", "Valor do Risco de Fogo previsto para o dia da detecção do foco. Valores inválidos -999 foram setados para 0."),
    ("bioma", "STRING", "Nome do Bioma segundo referência do IBGE 2004. Para outros países o campo pode ficar nulo (representado como ' ')."),
    ("frp", "DOUBLE", "Fire Radiative Power, MW (megawatts)."),
    ("data_ref_carga", "STRING", "Data do processamento da partição (YYYY-MM-DD)")
]

# descrição geral da tabela
descricao_tabela = (
    "Tabela Silver - Focos de Queimadas e Incêndios (tratada)."
)

In [0]:
# ==============================
# CRIAR TABELA SE NÃO EXISTIR (COM OS COMENTÁRIOS) OU SOBRESCREVER DADOS SE EXISTIR
# ==============================
from pyspark.sql.utils import AnalysisException

def create_table_with_comments(table_fqdn, columns_def, partition_col, table_description):
    # monta definição de colunas para CREATE TABLE
    cols_sql = ",\n  ".join([f"{name} {dtype} COMMENT '{comment}'" for name, dtype, comment in columns_def])
    sql = f"""
    CREATE TABLE {table_fqdn} (
      {cols_sql}
    )
    USING DELTA
    PARTITIONED BY ({partition_col})
    """
    # criar tabela
    spark.sql(sql)
    # setar descrição
    spark.sql(f"ALTER TABLE {table_fqdn} SET TBLPROPERTIES (description = '{table_description}')")

In [0]:
# verifica existência
table_exists = spark.catalog._jcatalog.tableExists(tabela_silver)  # uso interno catalog para suportar catalog.schema.table

if not table_exists:
    print(f"Tabela {tabela_silver} não existe. Criando com comentários...")
    try:
        create_table_with_comments(tabela_silver, columns_definition, "data_ref_carga", descricao_tabela)
    except Exception as e:
        raise RuntimeError(f"Erro ao criar tabela silver {tabela_silver}: {e}")
else:
    print(f"Tabela {tabela_silver} já existe. Irei gravar dados e atualizar comentários das colunas (se necessário).")

In [0]:
# ==============================
# GRAVANDO DADOS NA TABELA SILVER (por partição)
# - Usamos overwrite forçando apenas a partição específica para evitar perder histórico
# ==============================
try:
    # opção: sobrescrever apenas a partição específica (modo compatível com Delta)
    (
        df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .partitionBy("data_ref_carga")
        .saveAsTable(tabela_silver)
    )
    print("Gravação realizada com sucesso na tabela silver.")
except Exception as e:
    raise RuntimeError(f"Erro ao gravar na tabela silver {tabela_silver}: {e}")

In [0]:
# ==============================
# FIM - resumo / contagem de registros
# ==============================
record_count = df.count()
print(f"Registros processados para data_ref_carga={data_ref_carga}: {record_count}")
print("Job finalizado com sucesso ✅")